In [ ]:
from wavelet1d import *

In [ ]:
d = 3
d_t = 5

# Primal MRA

In [ ]:
mra = PrimalMRA(d)

In [ ]:
print(mra.ML)
if d == 2:
    ML_ref = np.array([[1],[1/2]])
    assert np.allclose(mra.ML, ML_ref)
if d == 3:
    ML_ref = np.array([[1, 0], [1/2, 1/2], [0, 3/4], [0, 1/4]])
    assert np.allclose(mra.ML, ML_ref)

- Primbs - 2006, Beispiel 3.18:  
    For $d = 2$, $M^L = [[1], [1/2]]$  
    For $d = 3$, $M^L = [[1, 0], [1/2, 1/2], [0, 3/4], [0, 1/4]]$

In [ ]:
j = 5
M0 = mra.refinement_matrix(j)
plt.rcParams['figure.figsize'] = [20, 10]
plt.spy(M0, marker='o', markersize=5)
plt.show()

- Primbs - 2006, Abbildung 3.5

In [ ]:
j = 3
plt.rcParams['figure.figsize'] = plt.rcParamsDefault['figure.figsize']
mra.plot(j)

- Primbs - 2006, Abbildung 3.2-3.4

In [ ]:
# basis functions constructed using refinement relation
mra.plot(j, from_refine_mat=True)